In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import ViTHybridImageProcessor, ViTHybridForImageClassification
from transformers import AutoImageProcessor, SwinForImageClassification
from PIL import Image
import requests
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from sklearn.metrics import roc_auc_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from scipy.optimize import brentq

Matplotlib is building the font cache; this may take a moment.


# Import Visual transformer model from https://huggingface.co/microsoft/swin-tiny-patch4-window7-224

In [2]:

image_processor = AutoImageProcessor.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
model = SwinForImageClassification.from_pretrained("microsoft/swin-tiny-patch4-window7-224")


preprocessor_config.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

C:\Users\nguyenl37\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nguyenl37\.cache\huggingface\hub\models--microsoft--swin-tiny-patch4-window7-224. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/113M [00:00<?, ?B/s]

In [3]:
# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]

        if self.transform:
            sample = self.transform(sample)

        return sample, label

In [ ]:
pwd

In [9]:

# Define data transforms (you can modify these based on your needs)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a consistent size
    transforms.ToTensor(),           # Convert images to tensors
])

# Define paths to your data folders
# train_data_dir = '/u/45/muhammu2/data/Desktop/face/Evaluation/CASIA_dataset/training set/'
# val_data_dir = '/u/45/muhammu2/data/Desktop/face/Evaluation/CASIA_dataset/testing set/'
# test_data_dir = '/u/45/muhammu2/data/Desktop/face/Evaluation/REPLAY_ATTACK/test/'

# Custom ImageFolder class to handle custom class names
class CustomImageFolder(ImageFolder):
    def find_classes(self, directory):
        classes = ['attack1', 'real1']
        class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
        return classes, class_to_idx
train_data_dir = 'c:/Users/nguyenl37/OneDrive - Aalto University/University/First year/Project/Face_Morphing_Attack_Detection/Casia_dataset/train/'
val_data_dir = 'c:/Users/nguyenl37/OneDrive - Aalto University/University/First year/Project/Face_Morphing_Attack_Detection/Casia_dataset/test/'
test_data_dir = 'c:/Users/nguyenl37/OneDrive - Aalto University/University/First year/Project/Face_Morphing_Attack_Detection/Replay Attack dataset/test/'


# Load your training, validation, and test datasets using ImageFolder
# train_dataset = ImageFolder(root=train_data_dir, transform=transform)
# val_dataset = ImageFolder(root=val_data_dir, transform=transform)
# test_dataset = ImageFolder(root=test_data_dir, transform=transform)

train_dataset = CustomImageFolder(root=train_data_dir, transform=transform)
val_dataset = CustomImageFolder(root=val_data_dir, transform=transform)
test_dataset = CustomImageFolder(root=test_data_dir, transform=transform)

# If you want to access the labels for train, validation, and test datasets:
train_labels = train_dataset.targets
val_labels = val_dataset.targets
test_labels = test_dataset.targets

# Create data loaders
batch_size = 8  # Adjust the batch size as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [10]:
# Define a binary classification head
class BinaryClassificationHead(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(BinaryClassificationHead, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Modify the Swin Transformer model for binary classification
classifier_head = BinaryClassificationHead(768, 32)  # Adjust input size as needed
model.classifier = classifier_head

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)


In [11]:
import numpy as np
import torch
from sklearn.metrics import roc_curve

# Assuming model, train_loader, val_loader, test_loader, criterion, and optimizer are defined

num_epochs = 50
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Initialize early stopping parameters
patience = 5
verbose = True
delta = 0.001  # For validation loss improvements
best_val_loss = float('inf')
counter = 0
best_eer_threshold = 0.5  # Placeholder for the EER threshold from the validation set

# Initialize lists to store training and validation losses
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    # Training loop
    for data, labels in train_loader:
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(data).logits  # Get logits from model
        loss = criterion(outputs, labels.unsqueeze(1).float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0.0
    val_labels = []  # Store true labels
    val_scores = []  # To store probabilities for EER calculation
    with torch.no_grad():
        for val_data, val_labels_batch in val_loader:
            val_data, val_labels_batch = val_data.to(device), val_labels_batch.to(device)
            val_outputs = model(val_data).logits
            val_loss += criterion(val_outputs, val_labels_batch.unsqueeze(1).float()).item()
            
            # Get probabilities (sigmoid for binary classification)
            val_probs = torch.sigmoid(val_outputs).cpu().numpy()  # Convert logits to probabilities
            val_scores.extend(val_probs)
            val_labels.extend(val_labels_batch.cpu().numpy())

    # Calculate average training and validation loss
    avg_train_loss = running_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")

    # EER Calculation on validation set
    fpr, tpr, thresholds = roc_curve(val_labels, val_scores, pos_label=1)  # Ensure pos_label is set
    fnr = 1 - tpr  # Calculate False Negative Rate
    eer_threshold_idx = np.nanargmin(np.abs(fnr - fpr))  # Find index where FNR == FPR
    eer_threshold = thresholds[eer_threshold_idx]  # EER threshold
    eer_fpr = fpr[eer_threshold_idx]
    eer_fnr = fnr[eer_threshold_idx]

    print(f"Epoch {epoch+1}, EER: {eer_fpr:.4f}, EER Threshold: {eer_threshold:.4f}")

    # Early stopping based on validation loss
    if avg_val_loss < best_val_loss - delta:
        best_val_loss = avg_val_loss
        best_eer_threshold = eer_threshold  # Save the EER threshold for the test set
        counter = 0
        torch.save(model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        counter += 1
        if counter >= patience:
            if verbose:
                print(f"Early stopping at epoch {epoch+1}")
            break

    # Append losses for plotting
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)


Epoch 1, Training Loss: 0.9455, Validation Loss: 1.0344
Epoch 1, EER: 0.5593, EER Threshold: 0.7167
Epoch 2, Training Loss: 0.7960, Validation Loss: 0.6475
Epoch 2, EER: 0.4704, EER Threshold: 0.4522
Epoch 3, Training Loss: 0.7188, Validation Loss: 0.5745
Epoch 3, EER: 0.5963, EER Threshold: 0.3211
Epoch 4, Training Loss: 0.6469, Validation Loss: 0.5645
Epoch 4, EER: 0.4519, EER Threshold: 0.2792
Epoch 5, Training Loss: 0.6203, Validation Loss: 0.5643
Epoch 5, EER: 0.4593, EER Threshold: 0.2781
Epoch 6, Training Loss: 0.6048, Validation Loss: 0.5634
Epoch 6, EER: 0.4889, EER Threshold: 0.2700
Epoch 7, Training Loss: 0.5990, Validation Loss: 0.5630
Epoch 7, EER: 0.4667, EER Threshold: 0.2657
Epoch 8, Training Loss: 0.5949, Validation Loss: 0.5629
Epoch 8, EER: 0.4889, EER Threshold: 0.2650
Epoch 9, Training Loss: 0.5933, Validation Loss: 0.5628
Epoch 9, EER: 0.4556, EER Threshold: 0.2639
Epoch 10, Training Loss: 0.5918, Validation Loss: 0.5628
Epoch 10, EER: 0.4926, EER Threshold: 0.264

## Training completed and best EER threshold saved from validation

In [13]:
from sklearn.metrics import roc_auc_score

# --- Now calculate HTER on the test set using the EER threshold from validation ---
test_labels = []
test_scores = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Get model predictions
        outputs = model(inputs)  # Get model predictions (SwinImageClassifierOutput)
        scores = torch.sigmoid(outputs.logits).cpu().numpy()  # Access logits and compute probabilities

        test_labels.extend(labels.cpu().numpy())  # Collect true labels
        test_scores.extend(scores)  # Collect predicted scores (probabilities)

# Classify test set based on the EER threshold from validation
false_accepts = 0  # Counter for false accepts (impostor samples classified as genuine)
false_rejects = 0  # Counter for false rejects (genuine samples classified as impostors)
total_genuine = 0  # Counter for total genuine samples
total_impostors = 0  # Counter for total impostor samples

# Iterate over the test labels and scores to calculate false accepts and rejects
for label, score in zip(test_labels, test_scores):
    if label == 0:  # Genuine sample
        total_genuine += 1
        if score >= best_eer_threshold:  # False rejection (genuine classified as impostor)
            false_rejects += 1
    else:  # Impostor sample
        total_impostors += 1
        if score < best_eer_threshold:  # False acceptance (impostor classified as genuine)
            false_accepts += 1

# Calculate FAR and FRR while avoiding division by zero
far = false_accepts / total_impostors if total_impostors > 0 else 0  # False Acceptance Rate
frr = false_rejects / total_genuine if total_genuine > 0 else 0  # False Rejection Rate

# Calculate HTER on the test set
hter = (far + frr) / 2  # Half Total Error Rate

# Output the results
print(f"Half Total Error Rate (HTER) on test set using EER threshold: {hter * 100:.2f}%")

# Calculate AUC on test set
auc = roc_auc_score(test_labels, test_scores)
print(f"Area Under the ROC Curve (AUC) on test set: {auc * 100:.2f}%")


Half Total Error Rate (HTER) on test set using EER threshold: 50.00%
Area Under the ROC Curve (AUC) on test set: 63.03%
